In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance
import random

pd.set_option('display.max_rows', 500)

In [11]:
objects = ['p', 's', 'g', 'c', 't']

coordinates = {'p': (0,1,3),
              's': (1,3,2),
              'g': (0,1,2.5),
               'c': (0,0,1),
               't': (1,3,2),
              'start': (0,1,0),
              'table': (1,2,2)}

c1 = {'p': 1.2,
    's': 1.2,
    'g': 1.2,
    'c': 1.2,
    't': 1.2}

k1 = {'p': 0.95,
    's': 1.0,
    'g': 1.0,
    'c': 1.0,
    't': 1.0}

c0 = {'p': 1.0,
    's': 1.0,
    'g': 1.0,
    'c': 1.0,
    't': 1.0}

k0 = {'p': 1.0,
    's': 1.0,
    'g': 1.0,
    'c': 1.0,
    't': 1.0}

start_coordinates = [[0,1,2],[1,1,2],[1,1,2],[1,1,2],[1,1,2]]

In [3]:
def predict_sequence(possible_items, coordinates, c, k, start_coordinates, dimensions=[3,]):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    
    if dimensions[0] == 3: # no changes if 3D
        coords = coordinates
        
    elif dimensions[0] == 2: # 2D: remove obsolete coordinate
        if dimensions[1] == 'xy':
            coords = {key: value[:-1] for key, value in coordinates.items()}
            for x in start_coords:
                x.pop(-1)
                
        elif dimensions[1] == 'xz':
            coords = {key: value for key, value in coordinates.items()}
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                coords[key] = new_value
            print(coords)
            for x in start_coords:
                x.pop(1)
        
        elif dimensions[1] == 'yz':
            coords = {key: value[1:] for key, value in coordinates.items()}
            for x in start_coords:
                x.pop(0)
                
    elif dimensions[0] == 1: # 1D: choose appropriate coordinate
        if dimensions[1] == 'x':
            coords = {key: value[0] for key, value in coordinates.items()}
            start_coords = [x[0] for x in start_coords]
        
        elif dimensions[1] == 'y':
            coords = {key: value[1] for key, value in coordinates.items()}
            start_coords = [x[1] for x in start_coords]
        
        elif dimensions[1] == 'z':
            coords = {key: value[2] for key, value in coordinates.items()}
            start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                start_coords[coord_index], 
                                coords[obj])
                                ) ** k[obj]) * c[obj]
        print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [12]:
predict_sequence(objects,coordinates, c1, k1, start_coordinates, dimensions=[3,'xyz'])

{'p': 1.2, 's': 2.6832815729997477, 'g': 0.6, 'c': 1.697056274847714, 't': 2.6832815729997477}
{'p': 1.6679018638108043, 's': 2.4, 'c': 2.0784609690826525, 't': 2.4}
{'s': 2.4, 'c': 2.0784609690826525, 't': 2.4}
{'s': 2.4, 't': 2.4}
{'t': 2.4}


['g', 'p', 'c', 's', 't']